<a href="https://colab.research.google.com/github/jumpingsphinx/loan_data/blob/main/data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
import seaborn as sns

In [ ]:
# Read the data
df = pd.read_csv(r"C:\Users\ashwi\Downloads\loan_data.csv")
df.head(10)

In [ ]:
# Process money amounts by stripping dollar signs, one-hot encode categorical variables
df['CHARGE_OFF_AMOUNT'] = df['CHARGE_OFF_AMOUNT'].str.replace('[\$,\s]', '', regex=True).astype(float)
df['DISBURSEMENT_AMOUNT'] = df['DISBURSEMENT_AMOUNT'].str.replace('[\$,\s]', '', regex=True).astype(float)
df['LOAN_AMOUNT'] = df['LOAN_AMOUNT'].str.replace('[\$,\s]', '', regex=True).astype(float)
df['IS_LOW_DOC'] = df['IS_LOW_DOC'].replace({'Y': 1, 'N': 0})
df['IS_NEW'] = df['IS_NEW'].replace({'Y': 1, 'N': 0})
df['APPROVAL_YEAR'] = df['APPROVAL_YEAR'].astype(str)
df['APPROVAL_YEAR'] = df['APPROVAL_YEAR'].str.replace(r'[^0-9]', '', regex=True).astype(int)

In [ ]:
def plot_3d_scatter(df, col1, col2, col3, sample_size=10000):
    """
    Create an interactive 3D scatter plot using Plotly.

    Parameters:
    df (pandas.DataFrame): Input DataFrame
    col1 (str): Name of the first column (x-axis)
    col2 (str): Name of the second column (y-axis)
    col3 (str): Name of the third column (z-axis)
    sample_size (int): Number of points to plot (default 10000)
    """
    # Sample the dataframe if it's larger than sample_size
    if len(df) > sample_size:
        df_sample = df.sample(n=sample_size, random_state=42)
    else:
        df_sample = df

    fig = px.scatter_3d(df_sample,
                        x=col1,
                        y=col2,
                        z=col3,
                        title=f'Interactive 3D Scatter Plot (Showing {sample_size:,} samples from {len(df):,} total points)')

    # Update marker size and opacity
    fig.update_traces(marker=dict(size=3,
                                opacity=0.6))

    # Improve the layout
    fig.update_layout(
        scene = dict(
            xaxis_title=col1,
            yaxis_title=col2,
            zaxis_title=col3
        )
    )

    fig.show()



def plot_2d_scatter(df, col1, col2, sample_size=10000):
    """
    Create an interactive 2D scatter plot using Plotly with sorted data.

    Parameters:
    df (pandas.DataFrame): Input DataFrame
    col1 (str): Name of the first column (x-axis)
    col2 (str): Name of the second column (y-axis)
    sample_size (int): Number of points to plot (default 10000)
    """
    # Sample the dataframe if it's larger than sample_size
    if len(df) > sample_size:
        df_sample = df.sample(n=sample_size, random_state=42)
    else:
        df_sample = df

    # Sort the dataframe by x-axis values
    df_sample = df_sample.sort_values(by=col1)

    fig = px.scatter(df_sample,
                    x=col1,
                    y=col2,
                    title=f'Loan Amount vs Charge Off Amount')

    # Update marker size and opacity
    fig.update_traces(marker=dict(size=5,
                                opacity=0.6))

    # Improve the layout
    fig.update_layout(
        xaxis_title=col1,
        yaxis_title=col2,
        width = 1300,
        height = 600
    )

    fig.show()

In [ ]:
# Can change edit this to be any column names in the data frame for visualization:
# plot_3d_scatter(df, col1, col2, col3):
# plot_2d_scatter(df, col1, col2)

In [ ]:
# Map industry code to the description of the industry
industry_mapping = {
    11: "Agriculture, forestry, fishing and hunting",
    21: "Mining, quarrying, and oil/gas",
    22: "Utilities",
    23: "Construction",
    31: "Manufacturing",  # Note: This covers 31-33
    42: "Wholesale trade",
    44: "Retail trade",   # Note: This covers 44-45
    48: "Transportation/warehousing", # Note: This covers 48-49
    51: "Information",
    52: "Finance and insurance",
    53: "Real estate",
    54: "Professional/scientific services",
    55: "Management of companies",
    56: "Administrative and support",
    61: "Educational services",
    62: "Healthcare and social assistance",
    71: "Arts and recreation",
    72: "Accommodation and food",
    81: "Other services",
    92: "Public administration"
}

# Create industry description column
df['INDUSTRY_DESC'] = df['INDUSTRY_ID'].astype(str).str[:2].astype(int).map(industry_mapping)

df_filtered = df[df['APPROVAL_YEAR'] >= 2008]

# Create the box plot
fig = px.box(df_filtered,
             x='INDUSTRY_DESC',
             y='CHARGE_OFF_AMOUNT',
             title='Distribution of Charge-off Amounts by Industry',
             labels={
                 'INDUSTRY_DESC': 'Industry',
                 'CHARGE_OFF_AMOUNT': 'Charge-off Amount ($)'
             })

# Customize the layout
fig.update_layout(
    showlegend=False,
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    height=800,  # Make plot taller to accommodate labels
    width=1200,
    margin=dict(b=150),  # Add bottom margin for rotated labels
    yaxis=dict(
        title_standoff=25,  # Add some space between y-axis title and values
        tickformat='$,.0f'  # Format y-axis as currency
    )
)
fig.show()

In [ ]:
def plot_industry_stats(df, year_cutoff=2008):
    """
    Calculate and display statistics for charge-off amounts by industry.

    Parameters:
    df (pandas.DataFrame): DataFrame containing loan data
    year_cutoff (int): Starting year for data analysis (default: 2008)

    Returns:
    pandas.DataFrame: DataFrame containing the calculated statistics
    """
    # Filter data
    df_filtered = df[df['APPROVAL_YEAR'] >= year_cutoff]

    # Calculate statistics by industry
    stats = df_filtered.groupby('INDUSTRY_DESC')['CHARGE_OFF_AMOUNT'].agg([
        ('count', 'count'),
        ('min', 'min'),
        ('q1', lambda x: x.quantile(0.25)),
        ('median', 'median'),
        ('q3', lambda x: x.quantile(0.75)),
        ('max', 'max'),
        ('mean', 'mean'),
        ('std', 'std')  # Added standard deviation
    ]).round(2)

    # Calculate IQR
    stats['iqr'] = stats['q3'] - stats['q1']

    # Reorder columns to a more logical sequence
    stats = stats[['count', 'min', 'q1', 'median', 'mean', 'q3', 'max', 'std', 'iqr']]

    # Sort by median charge-off amount to see industries from highest to lowest risk
    stats = stats.sort_values('median', ascending=False)

    # Format currency columns
    currency_cols = ['min', 'q1', 'median', 'mean', 'q3', 'max', 'std', 'iqr']
    for col in currency_cols:
        stats[col] = stats[col].apply(lambda x: f"${x:,.2f}")

    print(f"\nCharge-off Statistics by Industry ({year_cutoff}-Present):")
    print("\nNumber of entries per industry:")
    print(stats['count'].sort_values(ascending=False))
    print("\nDetailed statistics:")
    print(stats)

    return stats

In [ ]:
statistics = plot_industry_stats(df, year_cutoff=2008)

In [ ]:
def plot_loan_default_histogram(df):
    """
    Create a dual histogram comparing charge-off amounts for short vs long term loans,
    excluding the first bin.

    Parameters:
    df (pandas.DataFrame): Input DataFrame with TERM and charge-off amount columns
    """
    # Create loan category groups
    short_term = df[df['TERM'] < 60]['CHARGE_OFF_AMOUNT']
    long_term = df[df['TERM'] > 120]['CHARGE_OFF_AMOUNT']

    # Calculate the bin size to exclude the first bin
    max_value = max(short_term.max(), long_term.max())
    bin_size = max_value / 50  # Using 50 bins

    # Filter out values close to 0
    short_term = short_term[short_term > bin_size]
    long_term = long_term[long_term > bin_size]

    # Create the figure
    fig = go.Figure()

    # Add histogram traces for each loan category
    fig.add_trace(go.Histogram(
        x=short_term,
        name='Short Term (<60 months)',
        marker_color='red',
        opacity=0.75,
        nbinsx=100
        )
    )

    fig.add_trace(go.Histogram(
        x=long_term,
        name='Long Term (>120 months)',
        marker_color='blue',
        opacity=0.75,
        nbinsx=100
        )
    )

    # Update the layout
    fig.update_layout(
        title='Distribution of Charge-off Amounts: Short vs Long Term Loans',
        xaxis_title='Charge-off Amount',
        yaxis_title='Number of Loans',
        barmode='group',
        bargap=0.1,
        bargroupgap=0.1
    )


    # Force x-axis to start at first bin
    # fig.update_xaxes(range=[0, max_value])

    fig.show()

In [ ]:
plot_loan_default_histogram(df)

In [ ]:
# Create two groups for short-term and long-term loans
short_term = df[df['LOAN_AMOUNT'] < 1000000]['CHARGE_OFF_AMOUNT']
long_term = df[df['LOAN_AMOUNT'] > 1000000]['CHARGE_OFF_AMOUNT']

# Mann-Whitney U test
statistic, p_value = stats.mannwhitneyu(short_term, long_term, alternative='two-sided')
print(f'Mann-Whitney U test p-value: {p_value}')

# Also calculate medians for interpretation
print(f'Short-term median: {np.median(short_term)}')
print(f'Long-term median: {np.median(long_term)}')

# T-test
t_stat, t_p_value = stats.ttest_ind(short_term, long_term)
print(f't-test p-value: {t_p_value}')

# Kolmogorov-Smirnov test
ks_stat, ks_p = stats.ks_2samp(short_term, long_term)
print(f"Kolmogorov-Smirnov test p-value: {ks_p:.2e}")